In [1]:
import numpy as np
import pandas as pd
from pandas.testing import assert_index_equal

from scipy.stats import gumbel_r, multinomial, norm
import statsmodels.formula.api as sm
from statsmodels.sandbox.regression.gmm import GMM, IV2SLS

In [2]:
df = pd.read_csv("rcl_data_4.csv")
df.set_index(keys=['mktid','firmid', 'prodid'], inplace=True)
shares = df['share']

## Summary statistics

In [3]:
df

price     share      xvar      wvar
mktid firmid prodid                                        
1     1      5       2.606675  0.227237  2.202134 -0.273457
      2      7       3.783296  0.015946  2.755761  0.164740
             8       2.498175  0.175231  1.952670 -0.370695
      3      14      2.396051  0.149334  1.860606 -0.561297
      4      16      2.683033  0.179254  2.317975 -0.640421
...                       ...       ...       ...       ...
20    2      8       3.287951  0.062299  1.968643 -0.588434
      3      9       3.970653  0.030828  2.652435 -0.173701
      4      10      4.638089  0.001708  2.317975 -0.000259
             11      3.895897  0.000724  0.671772  0.914311
             12      4.020247  0.021632  2.660057  0.025131

[188 rows x 4 columns]

In [4]:
df_widxs = df.copy()
df_widxs['firmid'] = df_widxs.index.get_level_values('firmid')
df_widxs['prodid'] = df_widxs.index.get_level_values('prodid')
stats_mktlvl = df_widxs.groupby(by=['mktid']).aggregate({'firmid': 'count', 'prodid': 'count'})
stats_mktlvl.rename(columns={"firmid": "#Firms", "prodid": "#Products"}, inplace=True)

nfirms = stats_mktlvl["#Firms"]
nprods = stats_mktlvl["#Products"]

stats_mktlvl

,#Firms,#Products
mktid,,
1,7,7
2,7,7
3,7,7
4,11,11
5,11,11
6,8,8
7,6,6
8,12,12
9,10,10


In [5]:
df_std = df.groupby('mktid').std()
df_mean = df.groupby('mktid').mean()
df_min = df.groupby('mktid').min()
df_max = df.groupby('mktid').max()
stats_mktlvl.join(df_mean.join(df_std.join(df_min.join(df_max, rsuffix='_max'), rsuffix='_min'),rsuffix='_std',lsuffix='_mean'))

,#Firms,#Products,price_mean,share_mean,xvar_mean,wvar_mean,price_std,share_std,xvar_std,wvar_std,price_min,share_min,xvar_min,wvar_min,price_max,share_max,xvar_max,wvar_max
mktid,,,,,,,,,,,,,,,,,,
1,7,7,2.750983,0.118469,1.945542,-0.293838,0.504559,0.083901,0.645296,0.302619,2.310209,0.015946,0.671772,-0.640421,3.783296,0.227237,2.755761,0.164740
2,7,7,2.608820,0.136178,2.631723,-0.270162,0.705770,0.156169,0.432091,0.691993,1.733794,0.000308,1.857874,-1.286910,3.612128,0.409148,3.143323,0.703590
3,7,7,3.098984,0.128079,2.576645,0.065384,0.779273,0.144382,0.471870,0.526692,2.012440,0.001445,1.933339,-0.626037,4.166480,0.369714,3.143323,1.012786
4,11,11,3.530610,0.072886,1.983285,0.168326,0.857782,0.149572,0.740780,0.450978,2.483254,0.000044,0.671772,-0.721385,5.050753,0.498375,3.009212,0.795168
5,11,11,2.854352,0.082693,1.801961,-0.154398,0.846140,0.114486,0.667598,0.302468,1.442446,0.000100,0.671772,-0.612460,4.568714,0.367852,2.755761,0.250892
6,8,8,2.950200,0.106947,1.940615,0.029890,0.778380,0.133469,0.577514,0.636252,2.179758,0.000150,0.671772,-0.949792,4.542324,0.367265,2.660057,0.872742
7,6,6,2.868110,0.121251,1.825464,-0.006352,0.376994,0.066176,0.913509,0.716644,2.191228,0.028310,0.671772,-0.935112,3.203889,0.191099,2.755761,0.796828
8,12,12,3.062927,0.073310,2.144377,0.101264,0.587061,0.138590,0.603179,0.237462,1.890271,0.000790,0.779767,-0.139097,3.928379,0.491212,3.009212,0.699992
9,10,10,3.361858,0.076573,2.155827,0.141241,0.478922,0.119418,0.558861,0.296784,2.739443,0.000681,1.305570,-0.459850,4.237235,0.349676,3.143323,0.559578


## Logit model warmup

### 1. with OLS

To Do: add a constant in the dataset, as 2SLS does not add one by default

In [6]:
df["log_share"] = np.log(df["share"])
logit_ols = sm.ols(formula="log_share ~ price + xvar", data=df).fit()

In [7]:
print(logit_ols.summary())

                            OLS Regression Results                            
Dep. Variable:              log_share   R-squared:                       0.786
Model:                            OLS   Adj. R-squared:                  0.784
Method:                 Least Squares   F-statistic:                     340.1
Date:                Sat, 28 Jan 2023   Prob (F-statistic):           1.07e-62
Time:                        12:19:06   Log-Likelihood:                -274.26
No. Observations:                 188   AIC:                             554.5
Df Residuals:                     185   BIC:                             564.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.4678      0.360      4.073      0.0

### 2. With 2SLS

In [8]:
logit_2sls = IV2SLS(endog=df["log_share"], exog=df[["price", "xvar"]], instrument=df[["wvar", "xvar"]]).fit()
alpha, beta = logit_2sls.params

In [9]:
print(logit_2sls.summary())

                          IV2SLS Regression Results                           
Dep. Variable:              log_share   R-squared:                       0.940
Model:                         IV2SLS   Adj. R-squared:                  0.939
Method:                     Two Stage   F-statistic:                       nan
                        Least Squares   Prob (F-statistic):                nan
Date:                Sat, 28 Jan 2023                                         
Time:                        12:19:14                                         
No. Observations:                 188                                         
Df Residuals:                     186                                         
Df Model:                           2                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
price         -2.6853      0.219    -12.265      0.0

In [10]:
def get_logit_elasticities(mktid: int) -> pd.DataFrame:
    df_mkt = df[df.index.get_level_values('mktid')==mktid].droplevel('mktid')
    mkt_elst = -alpha*np.dot(np.ones((nfirms.loc[mktid],1)), (df_mkt["price"]*df_mkt["share"]).to_numpy().reshape((1,nfirms.loc[mktid]))) + alpha*np.diag(df_mkt["price"])
    return pd.DataFrame(mkt_elst, index=df_mkt.index, columns=df_mkt.index)

Matrix of own and cross-price elasticities for market 1:

In [11]:
get_logit_elasticities(1)

firmid                1         2                   3         4            \
prodid               5         7         8         14        16        17   
firmid prodid                                                               
1      5      -5.409185  0.162007  1.175525  0.960847  1.291500  0.283232   
2      7       1.590613 -9.997414  1.175525  0.960847  1.291500  0.283232   
       8       1.590613  0.162007 -5.532914  0.960847  1.291500  0.283232   
3      14      1.590613  0.162007  1.175525 -5.473355  1.291500  0.283232   
4      16      1.590613  0.162007  1.175525  0.960847 -5.913346  0.283232   
       17      1.590613  0.162007  1.175525  0.960847  1.291500 -7.717572   
       18      1.590613  0.162007  1.175525  0.960847  1.291500  0.283232   

firmid                   
prodid               18  
firmid prodid            
1      5       0.290840  
2      7       0.290840  
       8       0.290840  
3      14      0.290840  
4      16      0.290840  
       17      0.290840  
       18     -5.912846

In [12]:
def make_hausman_instr(mktid: int) -> pd.DataFrame:
    df_mkt = df[df.index.get_level_values('mktid')==mktid].droplevel('mktid')
    df_othermkts = df[~df.index.get_level_values('mktid').isin([mktid])]
    df_mkt["hausman_priceavg"] = df_othermkts.groupby(by=["firmid", "prodid"])["price"].mean()
    return df_mkt

In [13]:
df_mkt1 = make_hausman_instr(1)

In [14]:
df_mkt1

price     share      xvar      wvar  log_share  \
firmid prodid                                                      
1      5       2.606675  0.227237  2.202134 -0.273457  -1.481762   
2      7       3.783296  0.015946  2.755761  0.164740  -4.138520   
       8       2.498175  0.175231  1.952670 -0.370695  -1.741651   
3      14      2.396051  0.149334  1.860606 -0.561297  -1.901567   
4      16      2.683033  0.179254  2.317975 -0.640421  -1.718950   
       17      2.979443  0.035400  1.857874  0.055792  -3.341030   
       18      2.310209  0.046882  0.671772 -0.431524  -3.060126   

               hausman_priceavg  
firmid prodid                    
1      5               2.997245  
2      7               3.950095  
       8               2.964054  
3      14              3.679174  
4      16              3.114423  
       17              3.266178  
       18              2.422983

In [15]:
logit_haus_2sls = IV2SLS(endog=df_mkt1["log_share"], exog=df_mkt1[["price", "xvar"]], instrument=df_mkt1[["hausman_priceavg", "xvar"]]).fit()
alpha_haus, beta_haus = logit_haus_2sls.params
print(logit_haus_2sls.summary())

                          IV2SLS Regression Results                           
Dep. Variable:              log_share   R-squared:                       0.976
Model:                         IV2SLS   Adj. R-squared:                  0.967
Method:                     Two Stage   F-statistic:                       nan
                        Least Squares   Prob (F-statistic):                nan
Date:                Sat, 28 Jan 2023                                         
Time:                        12:19:47                                         
No. Observations:                   7                                         
Df Residuals:                       5                                         
Df Model:                           2                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
price         -1.6966      0.363     -4.672      0.0

/Users/slebst/Documents/georgetown/year2/struct-env/lib/python3.9/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 7 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


## Computing Instrumental Variables
 
### 1. BLP IVs

In [27]:
# Product attributes over all products in product space owned by same firm
blpiv_xvar_ownfirm = df['xvar'].groupby(by=["firmid", "prodid"]).sum()
# Use sum of product attributes over all firm and subtract own firm to get sum of attributes over all products not owned by firm
xvar_allfirms = df['xvar'].groupby("prodid").sum()
# From sum of attributes over all products not owned by firm, sum by firm
BLP_ivs = xvar_allfirms - blpiv_xvar_ownfirm

df = pd.merge(left=df, right=BLP_ivs, on=['firmid', 'prodid'], suffixes=["", "_BLP"])
df.rename(columns={"xvar_BLP": "blp_iv"}, inplace=True)

## Random Coefficients Logit Model

In [16]:
us_census = pd.read_csv("census-ageincome-joint-small.csv", index_col=0).astype(int)
census_density = (1./us_census.sum().sum())*us_census
n_age, n_inc = us_census.shape

In [17]:
def draw_from_census(n:int, normalize:bool=True) -> pd.DataFrame:
    census_distr = multinomial(n=1, p=census_density.to_numpy().flatten())
    census_draws = census_distr.rvs(n)
    dem_ids = np.array([np.argwhere(census_draws[k,:].reshape(n_age, n_inc))[0] for k in range(census_draws.shape[0])]).astype(float)
    if normalize:
        dem_ids[:,0] = dem_ids[:,0]/(n_age-1)
        dem_ids[:,1] = dem_ids[:,1]/(n_inc-1)
    return pd.DataFrame.from_dict({'age': dem_ids[:,0], 'income': dem_ids[:,1]})

def draw_from_normal(n: int) -> pd.DataFrame:
    return pd.DataFrame.from_dict({'age': norm.rvs(loc=0, scale=1, size=n), 'income': norm.rvs(loc=0, scale=1, size=n)})

def simulate_consumers(n:int=500, scale:float=1, sample_demographics:str='norm') -> pd.DataFrame:
    cons_df = pd.Series(data=gumbel_r.rvs(scale=scale, size=n), name='eps')
    if sample_demographics=='norm':
        demographics = draw_from_normal(n)
    elif sample_demographics=='census':
        demographics = draw_from_census(n)
    cons_df = pd.concat([cons_df, demographics], axis=1)
    return cons_df

In [18]:
cons = simulate_consumers()

In [ ]:
def MC_share_estimation(delta_est:pd.Series, mu_est:pd.DataFrame):
    """
    Implements a Monte-Carlo estimator of sigma_{jt}
    delta_est must be a Series indexed by the same MultiIndex((cdid,prodid)) as df
    """
    # assert_index_equal(delta_est.index, mu_est.index)

    normlzr = 1 + np.exp(mu_est.add(delta_est, axis='index')).groupby(by='mktid').sum()
    return np.exp(mu_est.add(delta_est, axis='index')).divide(
        normlzr,
        axis='index'
    ).mean(axis=1)


class NFP(GMM):
    """
    Implements the nested fixed-point method
    Arguments:
        mkt_df: pd.DataFrame at the mkt*prod*firm level containing data, including instruments to be used for 2SLS
        cons: pd.DataFrame containing consumer data
        instruments: list of field names from mkt_df to use as instruments in the 2SLS process
        theta2_0: initial guess of theta2
        tol_fp: tolerance on the variation on mean variation used to stop the fixed-point iteration
        tol_ol: tolerance on the variation of theta=(theta1, theta2) used to stop the outer-loop of the algorithm
    Returns:
        theta_1: np.array
        theta_2: np.array
        dmd_s
    """
    def __init__(self, mkt_df: pd.DataFrame, cons: pd.DataFrame, instruments: list[str], theta2_0: np.array, tol_fp: float=1e-6, tol_gmm: float=1e-6):
        endog = mkt_df["log_share"]
        exog = mkt_df[["price", "xvar"]]
        instrument = mkt_df[instruments + ["xvar"]]
        super(GMM).__init__(endog, exog, instrument)
        self.mkt_data = mkt_df
        self.cons_data  = cons
        self.theta2 = theta2_0
        self.tol_fp = tol_fp
        self.tol_gmm = tol_gmm

    def mean_val_guess(self, instrument='wvar'):
        """
        Returns a pd.Series at the scale of mkt*prod (no firms!)
        """
        delta_est = {}
        for mktprod, df_mktprod in self.mkt_data.groupby(by=['mktid', 'prodid']):
            alpha, beta = IV2SLS(endog=df_mktprod["log_share"], exog=df_mktprod[["price", "xvar"]], instrument=df_mktprod[["xvar", instrument]]).fit()
            delta_est[mktprod] = alpha*df_mktprod["price"]+beta*df_mktprod["xvar"]
        return pd.Series(delta_est)

    def idiosyncratic_coefs(self, theta2: np.array):
        return np.dot(self.cons_data[['age','income']].to_numpy(), theta2.reshape([2,1]))

    def idiosyncratic_valuations(self, theta2: np.array):
        self.cons_data["is_val_est"] = self.idiosyncratic_coefs(theta2)
        cust_vals = np.dot(self.mkt_data["xvar"],self.cons_data["is_val_est"].T)
        return pd.DataFrame(cust_vals, index=self.mkt_data.index, columns=["cust{0:d}".format(k) for k in range(self.cons_data["is_val_est"].shape[0])])

    def NFP_contr_map(self, expdel):
        return expdel*self.mkt_data["share"]/MC_share_estimation(np.log(expdel))

    def est_delta(self, delta0:pd.Series, tol:float=1e-6):
        """
        Iterates the contraction mapping until the error between shares and share estimates
        is smaller than tol, for the standard euclidean distance, with the sampling measure.
        """
        assert tol>0
        expdel = np.exp(delta0)

        while True:
            expdel_next = self.NFP_contr_map(expdel)
            err = np.linalg.norm(expdel_next - expdel, np.Inf)
            if err < tol:
                break
            expdel = expdel_next

        return np.log(expdel)

    def momcond(self, theta2: np.array):
        """
        Generates the empirical loss corresponding to the outer-loop GMM
        of the Nested-Fixed Point algorithm
        Arguments:
            theta_2: estimated value of idiosyncratic valuation parameters
        Returns:
            1. Empirical loss on moment conditions
            2. Estimation of coefficients for the mean valuation
            3. Estimation of demand shifters
        """
        # 1. From an estimate of theta2, recover the idiosyncratic valuations
        is_vals = self.idiosyncratic_valuations(theta2)

        # 2. From an initial guess of mean valuations from a "Step 0" IV, call the Fixed-Point procedure
        # until convergence to estimate mean valuations delta
        delta0 = self.mean_val_guess()
        delta = self.est_delta(delta0, tol=self.tol_fp)

        # 3. Use 2SLS to estimate theta1
        tsls = IV2SLS(endog=delta, exog=self.mkt_data[["price", "xvar"]], instrument=self.mkt_data[self.instrument + ["xvar"]]).fit()
        self.theta1_est = tsls.params
        self.dmd_shifters = tsls.resid

        #4. Recover empirical loss function
        return np.dot(self.dmd_shifters.T, self.mkt_data[self.instrument + ["xvar"]])

In [ ]:
def test_robustness(theta2_init_vals):
